# Ce notebook classe des phases en 3 catégories "cher", "moyen","éco".
IL fait partie d'une série de notebook permettant de comparer différentes approches de classification en phasant varié les architectures de réseaux de neurones et les méthodes d'encodification des textes.

Keras uses tensorflow uses sklearn uses numpy and pandas

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder


 DATA PROCESS
 les données ont été saisies sous excel et sauvegardées en csv utf-8 bizarement avec l'option séparateur virgules de microsoft
 


In [ ]:
def getDataset():
	dataset = pd.read_csv("RuleCout2.csv", delimiter=";", encoding='utf-8')
	#print ( 'dataset loaded with shape',dataset.shape)
	X = dataset['text']
	Y = dataset['class']
	dataset.set_index('text') #pour avoir les classes dans un ordre quelconque
	return dataset, X, Y

# utile pour les colonnes sur une ligne et afficher toutes les lignes
pd.set_option('display.width', 200)
pd.set_option('display.max_rows', 1000)

# on ajoutera peu à peu de nouvelles colonnes au dataset 
dataset, X, Y = getDataset()
print(dataset)

#### Les mots les plus signifiants ont été classés par catégorie.

La catégorie "éco" indique les mots qui vont dans le sens de choix économiques
La catégorie "troquet" indique des classes de restaurant
La catégorie "négation" indique des mots qui inverse le sens
La catégorie "éco" indique les mots qui vont dans le sens de choix économiques
La catégorie "monnaie" indique des quantités monayables
La catégorie "ambigu" contient des termes qui se caractérisent par leur absence de positionnenement qui est une information en soit
Les catégories "nombres" sont explicites.

Les mots les plus signifiants vont permettre un encdage par catégorie.
ansi "pas trop cher" deviendra "négation 0 cher" ou [5 0 10]. 
On se posera la question de la pérsence du zéro : est-elle bénéfique ou néfaste ?

#### Quel est l'impact de ce travail de répartition sur  le taux de réussite ?



In [ ]:
eco = ['modique','économique','ténu','japonais', 'chinois','kebab', 'self','pourri','avantageux','marché','bas',
       'plein','petit','petite']
troquet = ['troquet','bistrot','cantine','self','brasserie','restaurant','restau','resto']
negation = ['pas','ni','peu']
cher = ['cher','bon','haut','luxe','luxueux','fiche','fous','moyens', 'gastronomique', 'classe','riche','michelin','etoile','étoiles',
'standing','haut','chicos','onéreux','onereux','super''meilleur','meilleurs','grattin','gratin', 'bien']
monnaie = ['gamme', 'euro', 'prix', 'euros','problème','pb','qualité']
ambigu = ['milieu','moyen']
nombreEco = ['0','5','10','15','zéro','cinq','dix','quinze']
nombreCher= ['20','30','40','50','80','100','cent','vingt','trente','quarante','cinquante','quatre-vingt','cent']
limitatif = ['moins', 'peu','jusqu''à','plutôt']

# le choix de l'entier n'importe pas sauf pour le zéro qui est la valeur complémentée par pad_sequences
def score(word):
	if word in eco: 
		return .1
	elif word in nombreEco:
		return .2
	elif word in troquet: 
		return .25
	elif word in negation:
		return .4
	elif word in limitatif:
		return .41
	elif word in monnaie:
		return .5
	elif word in ambigu:
		return .51
	elif word in nombreCher:
		return .8
	elif word in cher:
		return .9
	else: 
		return 0.01
	
print ( 'score modique', score('modique'))
print ( 'score troquet', score('troquet'))
print ( 'score inconnu', score('inconnu'))


In [ ]:
#ai préféré utiliser le tokenizer de keras plutôt que celui de nltk car les valeurs filtrées par défaut comprennent l'apostrophe
def tokenize(sentence):
	tokens = text_to_word_sequence(sentence)
	return tokens
	
dataset['tokens']= dataset['text'].map(tokenize)
dataset['tokens']

L'étape suivante consiste à transformer une liste de tokens en une liste de nombres à encoder.
Comme la taille de cette liste doit être fixe on utilise pad_sequence qui ajoute des zéros ou enlève des nombres. 

In [ ]:
def token2code(tokens):
	return list(map(score,tokens))

#dataset['Xcodes'] = list (map(token2code,dataset['tokens']))
def encode(data):
	return dataset['tokens'].map(token2code)

Xcodes = encode(dataset)
XEncoded = pad_sequences(Xcodes,maxlen=5,padding='pre',dtype='float')
dataset['XEncoded'] = XEncoded.tolist()
dataset


Le prochain pavé permet de transformer les catégories "éco","moyen" et cher en vecteur.
Ces vecteurs sont rangés dans la colonne 'Ycodes' du dataset

In [ ]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
dataset['Ycodes'] = pd.Series(list(dummy_y))
dataset


###MODEL DEFINITION
Le réseau comprend trois couches.
La première couche doit correspondre à la taille du vecteur X i.e au nombre de dimensions ici c'est cinq.
La couche de sortie comprend 3 neurons avec une fonction d'activation softmax, cette répartition entre les trois poids indiquent une probabilité d'appartenir à une des trois classes.
La taille de couche du milieu est ici de 11 avec une fonction d'activation Relu.


In [ ]:
########################### MODEL DEFINITION
#def sequential_model():
#	# create model
#	model = Sequential()
#	model.add(Dense(5, input_dim=5, activation='relu'))
#	model.add(Dense(3, activation='softmax'))
#	# Compile model
#	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#	return model
max_features=5

def sequential_model():
    model = Sequential()
    model.add(Embedding(max_features, output_dim=5))
    model.add(LSTM(32))
    #model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
    return model

model = sequential_model()
model

In [ ]:
###### TRAINING + EVALUATION
x_train = XEncoded
y_train = dummy_y
batch_size = 5
model.fit(x_train, y_train,
          epochs=10,
          batch_size=batch_size)
x_test = x_train
y_test = y_train
loss,metric = model.evaluate(x_test, y_test, batch_size=batch_size)
print ("Loss = ", loss," Accuracy = ",metric)

In [ ]:
dataset['predicted'] = list(model.predict(XEncoded)) 

def decodeY(L):
	if L[0]>L[1] and L[0]>L[2]: 
		return "cher"
	elif L[2]>L[1] and L[2]>L[0]:
		return "éco"
	else:
		return "moyen"
	
dataset['resultat']=list(map(decodeY,dataset['predicted']))
dataset
